<a href="https://colab.research.google.com/github/Sasaki0Kojiro/hello-world/blob/main/pytorch1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

N,D_in,H,D_out = 64,1000,100,10

x = np.random.randn(N,D_in)
y = np.random.randn(N,D_out)

w1 = np.random.randn(D_in,H)
w2 = np.random.randn(H,D_out)

learning_rate = 1e-6
for t in range(500):
  h = x.dot(w1)
  h_relu = np.maximum(h,0)
  y_pred = h_relu.dot(w2)

  loss = np.square(y_pred - y).sum()
  if t%100 == 99:
    print(t,loss)

  grad_y_pred = 2.0 * (y_pred -y)
  grad_w2 = h_relu.T.dot(grad_y_pred)
  grad_h_relu = grad_y_pred.dot(w2.T)
  grad_h = grad_h_relu.copy()
  grad_h[h<0] = 0
  grad_w1 = x.T.dot(grad_h)

  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

99 679.9954044737376
199 5.905443712217223
299 0.11146833332255832
399 0.002709914912168784
499 7.550746009935352e-05


In [3]:
w1_unlearned = np.random.randn(D_in,H)
w2_unlearned = np.random.randn(H,D_out)

h = x.dot(w1_unlearned)
h_relu = np.maximum(h,0)
y_pred = h_relu.dot(w2_unlearned)

print(f"学習能力:{np.round(y_pred[0],decimals=2)}")

h = x.dot(w1)
h_relu = np.maximum(h,0)
y_pred = h_relu.dot(w2)
print(f"学習後出力:{np.round(y_pred[0],decimals=2)}")

print(f"目的の主力:{np.round(y[0],decimals=2)}")

学習能力:[ -14.27  381.42  333.79   34.87 -245.5    56.36 -259.05 -230.05 -214.99
   59.33]
学習後出力:[-1.29 -1.83  0.3  -0.07 -0.37  0.66 -1.17 -0.77  0.04 -0.03]
目的の主力:[-1.29 -1.83  0.3  -0.07 -0.37  0.66 -1.17 -0.77  0.04 -0.03]


In [8]:
import torch

dtype = torch.float
device = torch.device("cpu")
N,D_in,H,D_out = 64,1000,100,10

x = torch.randn(N,D_in,device=device,dtype = dtype)
y = torch.randn(N,D_out,device=device,dtype = dtype)

w1 = torch.randn(D_in ,H, device=device,dtype=dtype)
w2 = torch.randn(H,D_out,device=device,dtype=dtype)

In [9]:
learning_rate = 1e-6
for t in range(500):
  h = x.mm(w1)
  h_relu = h.clamp(min=0)
  y_pred = h_relu.mm(w2)

  loss = (y_pred - y).pow(2).sum().item()
  if t % 100 == 99:
    print(t,loss)
  
  grad_y_pred = 2.0 * (y_pred - y)
  grad_w2 = h_relu.t().mm(grad_y_pred)
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h<0] = 0
  grad_w1 = x.t().mm(grad_h)

  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

99 327.82122802734375
199 1.308321237564087
299 0.009671855717897415
399 0.00022688652097713202
499 3.5402867069933563e-05


In [12]:
import torch

dtype = torch.float
device = torch.device("cpu")

N,D_in,H,D_out = 64,1000,100,10

x = torch.randn(N,D_in, device=device,dtype=dtype,requires_grad=True)
y = torch.randn(N,D_out, device=device,dtype=dtype,requires_grad=True)

w1 = torch.randn(D_in, H , device = device,dtype = dtype,requires_grad=True)
w2 = torch.randn(H,D_out,device = device,dtype = dtype,requires_grad=True)

learning_rate = 1e-6
for t in range(500):
  #順伝播
  y_pred = x.mm(w1).clamp(min=0).mm(w2)

  #損失の計算と表示
  loss = (y_pred - y).pow(2).sum()
  if t % 100 == 99:
    print(t,loss.item())
  #逆伝播
  loss.backward()

　#確率的勾配降下法
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad
    #手動で勾配0
    w1.grad.zero_()
    w2.grad.zero_()

99 654.0030517578125
199 5.085948467254639
299 0.0664990246295929
399 0.0014155146200209856
499 0.00013018395111430436


In [13]:
import torch

In [14]:
class MyReLU(torch.autograd.Function):
  @staticmethod

  def forward(ctx,input):
    ctx.save_for_backward(input)
    return input.clamp(min = 0)

  @staticmethod

  def backward(ctx,grad_output):
    input, = ctx.saved_tensors
    grad_input = grad_output.clone()
    grad_input[input < 0 ] = 0
    return grad_input

In [15]:
dtype = torch.float
device = torch.device("cpu")

N,D_in,H,D_out = 64,1000,100,10
# 乱数により入力データと目標となる出力データを表すTensorを生成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 乱数による重みを表すTensorの定義
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

for t in range(500):
  #関数を適用するためにapplyメソッドをつかう
  relu = MyReLU.apply

  #順伝播
  y_pred = relu(x.mm(w1)).mm(w2)

  #損失
  loss = (y_pred - y).pow(2).sum()
  if t % 100 == 99:
    print(t,loss.item())
  
  #逆伝播
  loss.backward()

  #確率的勾配降下法
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    w1.grad.zero_()
    w2.grad.zero_()

99 488.1488952636719
199 2.781280279159546
299 0.024670353159308434
399 0.0004776020650751889
499 6.487738573923707e-05
